In [1]:
import pandas as pd

from IPython.display import display
from __future__ import unicode_literals

pd.set_option('expand_frame_repr', False)
pd.set_option('display.max_rows', 50)

In [2]:
profile = pd.read_csv('rp.session.two.jdakka.017398.0008.prof', 
                      header=None,
                      names=['tstamp','sid','uid','state','event','msg'],
                      usecols=['tstamp','sid','uid','state'])

# Keep only unit profiles
profile = profile.dropna(subset=['uid'])
profile = profile[profile.uid.str.contains('unit')]

# Elminate redundant 'Done' state 
profile.loc[profile.state == 'Done'] = profile.loc[(profile.sid.str.contains('OutputFileTransfer')) & 
                                                   (profile.state == 'Done')]
profile = profile.dropna()
if profile[profile.state == 'Done'].state.size != 112:
    print 'ERROR'

# Purge useless info from session ID
profile['sid'] = profile['sid'].apply(lambda x: x.split(':')[1])
profile = profile.reset_index(drop=True)

# Units Execution time ($T_x$)

In [3]:
# Keep only execution-related states
txs = profile[(profile.state == 'Executing') | 
              (profile.state == 'StagingOutput') | 
              (profile.state == 'AgentStagingOutputPending')].copy()

# Profiles are a mess in 0.42 :( 
# - Duplicates of state StagingOutput 
# - spare presence of state AgentStagingOutputPending
for uid in txs.uid.tolist():
    txs[txs.uid == uid]
    if len(txs[(txs.uid == uid) & (txs.state == 'StagingOutput')]['state'].tolist()) >= 2:
        txs = txs.drop(txs.index[(txs.uid == uid) &
                                 (txs.state == 'StagingOutput') & 
                                 (txs.sid.str.contains('Thread'))])
    if 'AgentStagingOutputPending' in txs[txs.uid == uid].state.tolist() and \
       'StagingOutput' in txs[txs.uid == uid].state.tolist():
        txs = txs.drop(txs.index[(txs.uid == uid) & 
                                  (txs.state == 'AgentStagingOutputPending')])

# We are done with sid
txs = txs.drop('sid', axis=1)

# Calculate $T_x$
txs.tstamp = pd.to_numeric(txs.tstamp, errors='coerce')
txs = txs.pivot(index='uid', columns='state', values='tstamp')
txs['Tx'] = txs['StagingOutput']-txs['Executing']
txs

state,Executing,StagingOutput,Tx
uid,,,
unit.000000,1571.5863,2009.2508,437.6645
unit.000001,1571.5895,2009.8600,438.2705
unit.000002,1571.5871,2010.2577,438.6706
unit.000003,1571.5887,2009.6564,438.0677
unit.000004,1571.5884,2010.0730,438.4846
unit.000005,1571.5882,2009.4418,437.8536
unit.000006,1571.5893,2010.0530,438.4637
unit.000007,1571.5878,2000.0152,428.4274
unit.000008,1571.5889,2014.6854,443.0965
